In [53]:
import pandas as pd
import os
import nltk
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thaveesha/Developer/projects/PsyVBot/nltk_data.
[nltk_data]     ..
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thaveesha/Developer/projects/PsyVBot/nltk_data.
[nltk_data]     ..
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thaveesha/Developer/projects/PsyVBot/nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
df = pd.read_csv('../datasets/depression-tweets.csv', encoding='latin-1')
df.columns = ['score', 'unknown2', 'date', 'query', 'username', 'text']
df.head()

,score,unknown2,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [45]:
df.columns

Index(['score', 'unknown2', 'date', 'query', 'username', 'text'], dtype='object')

In [46]:
df.dtypes

score        int64
unknown2     int64
date        object
query       object
username    object
text        object
dtype: object

In [47]:
df.shape

(1599999, 6)

In [50]:
df.score.unique()

array([0, 4])

In [51]:
df = df.drop(['unknown2', 'date', 'query', 'username'], axis = 'columns')
df.head()

,score,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [54]:
def remove_punctuation(text):
    # check if the input is a string
    if isinstance(text, str):
        # create a translation table with punctuation marks mapped to None
        translator = str.maketrans('', '', string.punctuation)
        # remove punctuation using the translation table
        return text.translate(translator)
    else:
        # if the input is not a string, return the input unchanged
        return text

# apply the remove_punctuation function to the 'A' column
df['text'] = df['text'].apply(lambda x: remove_punctuation(x))
df.head()

,score,text
0,0,is upset that he cant update his Facebook by t...
1,0,Kenichan I dived many times for the ball Manag...
2,0,my whole body feels itchy and like its on fire
3,0,nationwideclass no its not behaving at all im ...
4,0,Kwesidei not the whole crew


In [55]:
# Tokenize sentences into words
df['text'] = df['text'].apply(lambda x: word_tokenize(x.lower()))

# Remove stop words
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

# Apply Lemmatization
lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

df.tail()

,score,text
1599994,4,"[woke, school, best, feeling, ever]"
1599995,4,"[thewdbcom, cool, hear, old, walt, interview, ..."
1599996,4,"[ready, mojo, makeover, ask, detail]"
1599997,4,"[happy, 38th, birthday, boo, alll, time, tupac..."
1599998,4,"[happy, charitytuesday, thenspcc, sparkscharit..."


In [56]:
df.to_csv('../datasets/cleaned_twitter_dataset.csv', header=False, encoding='utf-8', index=False)
df = pd.read_csv('../datasets/cleaned_twitter_dataset.csv')
df.columns = ['score', 'text',]
df.tail()

,score,text
1599993,4,"['woke', 'school', 'best', 'feeling', 'ever']"
1599994,4,"['thewdbcom', 'cool', 'hear', 'old', 'walt', '..."
1599995,4,"['ready', 'mojo', 'makeover', 'ask', 'detail']"
1599996,4,"['happy', '38th', 'birthday', 'boo', 'alll', '..."
1599997,4,"['happy', 'charitytuesday', 'thenspcc', 'spark..."
